## 出租车数据预处理

In [1]:
FILES_DIR = {
    # 文件夹
    "taxi_dir": "E:\MSE\VAG\VIS2021\温州数据\温州出租车数据\TRK20140101",
#     "taxi_dir": "/Volumes/Age's SSD 1/SSD/数据/出租车/201401TRK/TRK20140101",
    "taxi_sample": "/Volumes/Age's SSD 1/SSD/数据/出租车/201401TRK/TRK20140107/C03/浙C03860.LOG"
}

In [2]:
import pandas as pd
import json

In [3]:
from struct import *
import time

def inRange(area,point):
    return point['longitude']>area["minLng"] and point['longitude']<area["maxLng"] and point["latitude"]>area["minLat"] and point["latitude"]<area["maxLat"]

def read_taxi_raw_file(area,filename):
    fmt = '>ffQ'
    ps = []
    with open(filename, "rb") as f:
        line = f.read(39)
        while(line):
            s = line[0:16]
            # 读取二进制文件 https://docs.python.org/zh-cn/3/library/struct.html
            data = unpack(fmt, s)
            point = {
                "longitude": data[0],
                "latitude": data[1]
            }
            if inRange(area,point):
                point["time"]=time.strftime("%Y-%m-%d %H:%M:%S",  time.localtime(data[2]/1000))
                ps.append(point)
            line = f.read(39)
    return ps

# 读取 Log 文件到 Dataframe
def read_taxi_df(area,filename):
    points = read_taxi_raw_file(area,filename)
    df_taxi = pd.DataFrame(points)
    return df_taxi
# read_taxi_df(FILES_DIR["taxi_sample"])

In [4]:
import json

def pack_one_taxi_to_json(df, carNo):
    points_str = df.to_json(orient='records')
    points = json.loads(points_str)
    taxi_trajectory = {
        "carNo": carNo,
        "points": points
    }
    return taxi_trajectory
    # file_name = carNo + '.json'
# j = pack_one_taxi_to_json(df_taxi_points, carNo)
# print(json.dumps(j, indent=4))

def write_json_to_file(json_obj, filename='taxi.json'):
    path = '../data_in_use/'
    filename = path + filename
    with open(filename, 'w') as outfile:
        json.dump(json_obj, outfile)

def write_json_to_node_server_file(json_obj, filename='taxi.json'):
    path = '../node-server-data/'
    filename = path + filename
    with open(filename, 'w') as outfile:
        json.dump(json_obj, outfile)

In [5]:
from os import walk
import util
# import json
# 获取文件列表
def list_files():
    raw_files = []
    BASE_PATH = FILES_DIR["taxi_dir"]
    for (dirpath, dirnames, filenames) in walk(FILES_DIR["taxi_dir"]):
        if(len(dirnames) == 0):
            for filename in filenames:
                if "._" not in filename:
                    raw_files.append(
                        [dirpath + '/' + filename, filename]
                    )
    # print(len(raw_files))
    return raw_files

# 读取原始文件 并转换至 json 格式
def read_files(area,num = 20):
    files = list_files()
    jsons = []
    c = 0
    for [f, carNo] in files:
        # print('doing ' + carNo + '...')
        c = c + 1
        df = read_taxi_df(area,f)
        taxi_json = pack_one_taxi_to_json(df, carNo)
        jsons.append(taxi_json)
        if c >= num:
            break
    write_json_to_file(jsons, 'taxi_'+ str(num) + '.json')
    print('done !')

# area = {
#         "minLat" : 27.958246,
#         "maxLat" : 28.027669,
#         "minLng" : 120.593029,
#         "maxLng" : 120.747524
# } 
config = util.getJsonData('./config.json')
read_files(config["area"],1000)

done !


#### VAUD  Case 部分

### 1. 找到两辆目标出租车的轨迹

In [6]:
cars = ['0230', '1168']

def find_cars_with_id(targetCarIds):
    files = list_files()
    carFiles = []
    for file in files:
        carNo = file[1]
        for carId in targetCarIds:
            if(carNo.find(carId) != -1):
                carFiles.append(file)
                break
    # print(carFiles)
    for [f, carNo] in carFiles:
        df = read_taxi_df(f)
        taxiJson = pack_one_taxi_to_json(df, carNo)
        write_json_to_node_server_file(taxiJson, 'taxi_'+ carNo + '.json')
        print('Write Car ' + carNo + ' Done!')
    
# find_cars_with_id(cars)